<a href="https://colab.research.google.com/github/alorenz465446/NYC_Parking_Tickets/blob/main/NYC_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import dependencies
import pandas as pd

In [2]:
# Add code to import from computer to google colab. 
from google.colab import files
filtered_df = files.upload()

Saving dropped_na_nyc_filtered.csv to dropped_na_nyc_filtered (3).csv


In [3]:
nyc_df = pd.read_csv("dropped_na_nyc_filtered.csv", low_memory=False)
nyc_df 

,Plate_Type,Violation_Code,Vehicle_Body_Type,Violation_Time,Violation_County,Vehicle_Color,Vehicle_Year
0,PAS,21.0,SUBN,0950A,K,SILVE,2008
1,PAS,38.0,SUBN,0817A,NY,WH,2014
2,PAS,21.0,4DSD,0148A,Q,BLACK,0
3,PAS,21.0,4DSD,0820A,K,RED,0
4,PAS,36.0,4DSD,1047A,QN,GY,2011
...,...,...,...,...,...,...,...
338518,PAS,21.0,SUBN,1201P,Q,GR,2004
338519,PAS,14.0,SUBN,1046A,Q,BLACK,0
338520,PAS,36.0,4DSD,0200P,QN,RD,2003
338521,PAS,38.0,4DSD,0202P,K,BL,2015


In [4]:
list(nyc_df.columns)

['Plate_Type',
 'Violation_Code',
 'Vehicle_Body_Type',
 'Violation_Time',
 'Violation_County',
 'Vehicle_Color',
 'Vehicle_Year']

In [5]:
nyc_df.dtypes

Plate_Type            object
Violation_Code       float64
Vehicle_Body_Type     object
Violation_Time        object
Violation_County      object
Vehicle_Color         object
Vehicle_Year           int64
dtype: object

In [12]:
# drop Violation_Time column
nyc_df = nyc_df.drop("Violation_Time", axis=1)

In [13]:
nyc_df['Vehicle_Color'].value_counts()

GREY      102581
BLACK      90550
WHITE      60510
BLUE       36250
RED        23533
SILVER     12674
GREEN      11477
YELLOW       508
ORANGE       230
PURPLE       210
Name: Vehicle_Color, dtype: int64

In [14]:
# Fix Colors 
def change_color(color):
  if color == 'GY':
    return 'GREY'
  elif color == 'BK':
    return 'BLACK'
  elif color == 'BLAK':
    return 'BLACK'
  elif color == 'WH':
    return 'WHITE'
  elif color == 'BL':
    return 'BLUE'
  elif color == 'RD':
    return 'RED'
  elif color == 'SILVE':
    return 'SILVER'
  elif color == 'GR':
    return 'GREEN'
  elif color == 'YW':
    return 'YELLOW'
  elif color == 'YELLO':
    return 'YELLOW'
  elif color == 'ORANG':
    return 'ORANGE'
  elif color == 'PURPL':
    return 'PURPLE'
  else: return color;

In [15]:
nyc_df['Vehicle_Color']= nyc_df['Vehicle_Color'].apply(change_color)
nyc_df

,Plate_Type,Violation_Code,Vehicle_Body_Type,Violation_County,Vehicle_Color,Vehicle_Year
0,PAS,21.0,SUBN,K,SILVER,2008
1,PAS,38.0,SUBN,NY,WHITE,2014
2,PAS,21.0,4DSD,Q,BLACK,0
3,PAS,21.0,4DSD,K,RED,0
4,PAS,36.0,4DSD,QN,GREY,2011
...,...,...,...,...,...,...
338518,PAS,21.0,SUBN,Q,GREEN,2004
338519,PAS,14.0,SUBN,Q,BLACK,0
338520,PAS,36.0,4DSD,QN,RED,2003
338521,PAS,38.0,4DSD,K,BLUE,2015


In [16]:
nyc_df['Vehicle_Color'].value_counts()

GREY      102581
BLACK      90550
WHITE      60510
BLUE       36250
RED        23533
SILVER     12674
GREEN      11477
YELLOW       508
ORANGE       230
PURPLE       210
Name: Vehicle_Color, dtype: int64

In [41]:
# checkout Violation_Code column 
nyc_df['Violation_Code'].value_counts()


21.0    101941
36.0     87992
38.0     70018
20.0     42380
14.0     36192
Name: Violation_Code, dtype: int64



*   21	Street Cleaning: No parking where parking is not allowed by sign, street marking or traffic control device.
*   36	Exceeding the posted speed limit in or near a designated school zone.
*   38	Failing to show a receipt or tag in the windshield.
*   20	General No Parking: No parking where parking is not allowed by sign, street marking or traffic control device.
*   14	General No Standing: Standing or parking where standing is not allowed by sign, street marking or; traffic control device.


# keep 38, bin 20 and 14 
# prediction 
no parking gets ticketed 




Consider Binning. 

Sample code below: 

```

# Look at INCOME_AMT value counts for binning
INCOME_AMT_count=application_df['INCOME_AMT'].value_counts()

# Visualize the value counts of INCOME_AMT
INCOME_AMT_count.plot.density()

---

# Determine which values to replace if counts are less than ...?
replace_INCOME_AMT = list(INCOME_AMT_count[INCOME_AMT_count < 500].index)


# Replace in dataframe
for app in replace_INCOME_AMT:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(app,"5M+")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()
```

# Encode Categorical Data 
Use one One_Hot_Encoder 

In [17]:
# Generate our categorical variable list
df_categorical = nyc_df.dtypes[nyc_df.dtypes == "object"].index.tolist()
df_categorical



['Plate_Type', 'Vehicle_Body_Type', 'Violation_County', 'Vehicle_Color']

In [19]:
nyc_df['Plate_Type'].value_counts()

PAS    333754
COM      4769
Name: Plate_Type, dtype: int64

In [18]:
# Create the OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(nyc_df[df_categorical]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(df_categorical)
encode_df.head()


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Plate_Type_COM,Plate_Type_PAS,Vehicle_Body_Type_2DSD,Vehicle_Body_Type_4DSD,Vehicle_Body_Type_PICK,Vehicle_Body_Type_SUBN,Violation_County_BK,Violation_County_BX,Violation_County_K,Violation_County_MN,...,Vehicle_Color_BLACK,Vehicle_Color_BLUE,Vehicle_Color_GREEN,Vehicle_Color_GREY,Vehicle_Color_ORANGE,Vehicle_Color_PURPLE,Vehicle_Color_RED,Vehicle_Color_SILVER,Vehicle_Color_WHITE,Vehicle_Color_YELLOW
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Merge one-hot encoded features and drop the originals
nyc_df = nyc_df.merge(encode_df,left_index=True, right_index=True)
nyc_df = nyc_df.drop(df_categorical,1)
nyc_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Violation_Code,Vehicle_Year,Plate_Type_COM,Plate_Type_PAS,Vehicle_Body_Type_2DSD,Vehicle_Body_Type_4DSD,Vehicle_Body_Type_PICK,Vehicle_Body_Type_SUBN,Violation_County_BK,Violation_County_BX,...,Vehicle_Color_BLACK,Vehicle_Color_BLUE,Vehicle_Color_GREEN,Vehicle_Color_GREY,Vehicle_Color_ORANGE,Vehicle_Color_PURPLE,Vehicle_Color_RED,Vehicle_Color_SILVER,Vehicle_Color_WHITE,Vehicle_Color_YELLOW
0,21.0,2008,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,38.0,2014,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,21.0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21.0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,36.0,2011,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


# Start with a Random Forest model to check for correlations. 

Note:
(used homework 17.8.2-3, named random_forest_model in JN)
- BM

***need to create a sampling of the data to use RandomForest

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


### Preprocess the Data


In [25]:
list(nyc_df.columns )


['Violation_Code',
 'Vehicle_Year',
 'Plate_Type_COM',
 'Plate_Type_PAS',
 'Vehicle_Body_Type_2DSD',
 'Vehicle_Body_Type_4DSD',
 'Vehicle_Body_Type_PICK',
 'Vehicle_Body_Type_SUBN',
 'Violation_County_BK',
 'Violation_County_BX',
 'Violation_County_K',
 'Violation_County_MN',
 'Violation_County_NY',
 'Violation_County_Q',
 'Violation_County_QN',
 'Violation_County_R',
 'Violation_County_ST',
 'Vehicle_Color_BLACK',
 'Vehicle_Color_BLUE',
 'Vehicle_Color_GREEN',
 'Vehicle_Color_GREY',
 'Vehicle_Color_ORANGE',
 'Vehicle_Color_PURPLE',
 'Vehicle_Color_RED',
 'Vehicle_Color_SILVER',
 'Vehicle_Color_WHITE',
 'Vehicle_Color_YELLOW']

In [28]:
# Define the features set.
X = nyc_df.copy()
X = X.drop(['Violation_Code'], axis=1)
X.head()


,Vehicle_Year,Plate_Type_COM,Plate_Type_PAS,Vehicle_Body_Type_2DSD,Vehicle_Body_Type_4DSD,Vehicle_Body_Type_PICK,Vehicle_Body_Type_SUBN,Violation_County_BK,Violation_County_BX,Violation_County_K,...,Vehicle_Color_BLACK,Vehicle_Color_BLUE,Vehicle_Color_GREEN,Vehicle_Color_GREY,Vehicle_Color_ORANGE,Vehicle_Color_PURPLE,Vehicle_Color_RED,Vehicle_Color_SILVER,Vehicle_Color_WHITE,Vehicle_Color_YELLOW
0,2008,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2014,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2011,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Define the target set.
y = nyc_df["Violation_Code"].ravel()
y[:5]


array([21., 38., 21., 21., 36.])

In [31]:
# Split data into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [32]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


### Fit the Random Forest Model


In [33]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128) 


> The `n_estimators` will allow us to set the number of trees that will be created by the algorithm. Generally, the higher number makes the predictions stronger and more stable, but can slow down the output because of the higher training time allocated. The best practice is to use between 64 and 128 random forests, though higher numbers are quite common despite the higher training time. 

In [34]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)


### Make Predictions Using the Testing Data


In [35]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions

array([36., 14., 36., ..., 21., 20., 21.])

### Evaluate the Model


21.0    101941
36.0     87992
38.0     70018
20.0     42380
14.0     36192

In [36]:
# # Calculating the confusion matrix.
# cm = confusion_matrix(y_test, predictions)

# # Create a DataFrame from the confusion matrix.
# cm_df = pd.DataFrame(
#     cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# cm_df


ValueError: ignored

In [37]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)


In [38]:
# Displaying results
# print("Confusion Matrix")
# display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("-")
print("Classification Report")
print(classification_report(y_test, predictions))


Accuracy Score : 0.5371672318653921
-
Classification Report
              precision    recall  f1-score   support

        14.0       0.29      0.06      0.10      9074
        20.0       0.22      0.01      0.03     10728
        21.0       0.43      0.79      0.56     25307
        36.0       0.86      0.95      0.91     22010
        38.0       0.34      0.21      0.26     17512

    accuracy                           0.54     84631
   macro avg       0.43      0.41      0.37     84631
weighted avg       0.48      0.54      0.47     84631



### Rank the Importance of Features


In [39]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances


array([0.09643971, 0.00161405, 0.00155458, 0.00151376, 0.00159074,
       0.00190965, 0.00184696, 0.2923248 , 0.03063194, 0.1022019 ,
       0.01085307, 0.07916191, 0.07685993, 0.22512771, 0.01041647,
       0.0501463 , 0.00150656, 0.00136522, 0.00153026, 0.00203808,
       0.00034313, 0.00039117, 0.00135042, 0.00475809, 0.00186281,
       0.00066079])

In [40]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)


[(0.2923247995798893, 'Violation_County_BK'),
 (0.22512770828443937, 'Violation_County_QN'),
 (0.10220189979876694, 'Violation_County_K'),
 (0.09643971447433264, 'Vehicle_Year'),
 (0.07916191400349451, 'Violation_County_NY'),
 (0.07685992942043036, 'Violation_County_Q'),
 (0.05014629511962221, 'Violation_County_ST'),
 (0.03063194056661346, 'Violation_County_BX'),
 (0.010853068695514358, 'Violation_County_MN'),
 (0.010416469776071961, 'Violation_County_R'),
 (0.00475809023862599, 'Vehicle_Color_SILVER'),
 (0.0020380799646986373, 'Vehicle_Color_GREY'),
 (0.001909650717123446, 'Vehicle_Body_Type_PICK'),
 (0.0018628079171640716, 'Vehicle_Color_WHITE'),
 (0.0018469596631573524, 'Vehicle_Body_Type_SUBN'),
 (0.0016140519272756892, 'Plate_Type_COM'),
 (0.001590738587456615, 'Vehicle_Body_Type_4DSD'),
 (0.0015545756972411085, 'Plate_Type_PAS'),
 (0.0015302644592650352, 'Vehicle_Color_GREEN'),
 (0.0015137575448518343, 'Vehicle_Body_Type_2DSD'),
 (0.0015065632491886873, 'Vehicle_Color_BLACK'),
 (

# Neuro Network 
We will be building a neuro network. First we need to clean the data, see correlations, and then make the plan for the neural network. 